In [87]:
import csv
import sys
import pandas as pd
import numpy as np
dataPath = "./INTEGRATED-DATASET.csv"



dataSet = pd.read_csv(dataPath)
dataSet.columns = ["Name", "Borough", "Type", "InspectionSeason", "Reason", "Critical", "Score", "Grade", "GradeSeason"]
dataSet
min_supp = 0.1*len(dataSet)
min_conf = 0.1*len(dataSet)



In [98]:
def get_L1(dataSet, one_item):
    for item in one_item:
        cntType = dataSet.groupby(item).count()
        one_item[item] = [x for x in one_item[item] if cntType.at[x, 'Name'] >= min_supp]

In [100]:
boroughs = dataSet["Borough"].unique()
grades = dataSet["Grade"].unique()
types = dataSet["Type"].unique()
gradeSeason = dataSet["GradeSeason"].unique()

gradeSeason = np.delete(gradeSeason, gradeSeason.shape[0]-1).tolist()
types = np.delete(types, types.shape[0]-1).tolist()
grades = np.delete(grades, grades.shape[0]-1).tolist()
boroughs = np.delete(boroughs, boroughs.shape[0]-1).tolist()
one_item={}
one_item["Type"] = types
one_item["Borough"] = boroughs
one_item["Grade"] = grades
one_item["GradeSeason"] = gradeSeason

get_L1(dataSet, one_item)
print (one_item)

{'Grade': ['A', 'B'], 'Borough': ['MANHATTAN', 'BROOKLYN', 'QUEENS'], 'Type': ['NorthAmerican', 'European', 'Asian', 'Other'], 'GradeSeason': ['Winter-GRADE DATE', 'Summer-GRADE DATE', 'Spring-GRADE DATE']}
